# Can you hear the shape of a drum? 

https://www2.math.upenn.edu/~kazdan/425S11/Drum-Gordon-Webb.pdf

You can't. This paper shows two different regions leading to the same spectrum. 
We numerically verify the result.

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [ ]:
shape1 = MoveTo(0,0).Line(1,1).Line(0,1).Line(-2,0).Line(0,1).Line(-1,-1).Line(1,-1).Line(1,0).Close().Face()
shape2 = MoveTo(0,0).Line(1,0).Line(0,1).Line(-2,2).Line(0,-1).Line(-1,0).Line(1,-1).Line(1,0).Close().Face()

In [ ]:
Draw (shape1)
Draw (shape2);

In [ ]:
def Calc(shape):
    shape.vertices.hpref=1
    mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.2))
    mesh.RefineHP(4)
    Draw(mesh)
    fes = H1(mesh, order=8, dirichlet=".*")
    u,v = fes.TnT()
    A = BilinearForm(grad(u)*grad(v)*dx).Assemble().mat
    M = BilinearForm(u*v*dx).Assemble().mat
    pre = A.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")
    num = 10
    evals, evecs = solvers.LOBPCG(A, M, pre=pre, num=num, maxit=20, printrates=False)  
    gfu = GridFunction(fes, multidim=num)
    for i in range(num):
        gfu.vecs[i] = evecs[i]
    return evals, gfu

In [ ]:
evals1, evecs1 = Calc(shape1)

In [ ]:
evals2, evecs2 = Calc(shape2)

In [ ]:
print ("eig1 = ", list(evals1))
print ("eig1 = ", list(evals2))

In [ ]:
Draw (evecs1, animate=True, min=-1, max=1);
Draw (evecs2, animate=True, min=-1, max=1);